# VAE (Variational AutoEncoder) Training

## imports

In [1]:
import os

from models.VAE import VariationalAutoencoder
from utils.loaders import load_mnist

Using TensorFlow backend.


In [2]:
# run params
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #

## data

In [3]:
(x_train, y_train), (x_test, y_test) = load_mnist()

## architecture

In [4]:
vae = VariationalAutoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

W0116 23:40:41.535077  4788 deprecation_wrapper.py:119] From c:\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0116 23:40:41.556973  4788 deprecation_wrapper.py:119] From c:\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0116 23:40:41.588885  4788 deprecation_wrapper.py:119] From c:\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0116 23:40:41.673658  4788 deprecation_wrapper.py:119] From c:\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.



In [5]:
vae.encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 28, 28, 32)   0           encoder_conv_0[0][0]             
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 14, 14, 64)   18496       leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
leaky_re_l

## The Encoder

**Example 7**: The variational autoencoder’s encoder.

<img src="./images/example_7.png" width="700" border="1">

## Lambda layer

A `Lambda` layer simply wraps any function into Keras layer. 

For example, the following layer squares its input: $$Lambda(lambda x: x ** 2)$$

They are useful when you want to apply a function to a tensor that isn’t already included as one of the out-of-the-box Keras layer types.

A diagram of the encoder is shown in **Figure 13**.

<img src="./images/figure_13.png" width="600" border="1">

**Figure 13**: Figure 3-13. Diagram of the VAE encoder.

In [6]:
vae.decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 64)        0         
__________

The decoder of a variational autoencoder is identical to the decoder of a plain autoencoder. 

The only other part we need to change is the **loss function**.

## The Loss Function

* In the code, the loss function for a VAE is simply the addition of the reconstruction loss and the KL divergence loss term. 


* We weight the reconstruction loss with a term, r_loss_factor, that ensures that it is well balanced with the KL divergence loss. 


* If we weight the reconstruction loss too heavily, the KL loss will not have the desired regulatory effect and we will see the same problems that we experienced with the plain autoencoder. 


* If the weighting term is too small, the KL divergence loss will dominate and the reconstructed images will be poor. 


* This weighting term is one of the parameters to tune when we are training your VAE.





**Example 8** shows how we include the KL divergence term in our loss function.

---
**Example 8**-Including KL divergence in the loss function:
<img src="./images/example_8.png" width="550" border="1">

## training

In [7]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 1000

In [8]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

W0116 23:40:52.736416  4788 deprecation_wrapper.py:119] From c:\anaconda3\envs\generative\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [9]:
BATCH_SIZE = 32
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [10]:
vae.train(     
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

W0116 23:40:58.206358  4788 deprecation_wrapper.py:119] From c:\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/200
60000/60000 [==============================] - 189s 3ms/step - loss: 58.5263 - vae_r_loss: 55.2357 - vae_kl_loss: 3.29061s - loss: 58.5463 - vae_r_loss: 55.2588 -

Epoch 00001: saving model to run/vae/0002_digits\weights/weights-001-58.53.h5

Epoch 00001: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 2/200
60000/60000 [==============================] - 184s 3ms/step - loss: 51.7331 - vae_r_loss: 47.7423 - vae_kl_loss: 3.9908

Epoch 00002: saving model to run/vae/0002_digits\weights/weights-002-51.73.h5

Epoch 00002: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 3/200
60000/60000 [==============================] - 182s 3ms/step - loss: 50.1700 - vae_r_loss: 45.8562 - vae_kl_loss: 4.3138

Epoch 00003: saving model to run/vae/0002_digits\weights/weights-003-50.17.h5

Epoch 00003: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 4/200
60000/60000 [==============================] - 183s 3ms/step - loss: 49.1672 - vae_r_loss: 44.6820 - va

60000/60000 [==============================] - 182s 3ms/step - loss: 45.3271 - vae_r_loss: 40.2117 - vae_kl_loss: 5.1154

Epoch 00030: saving model to run/vae/0002_digits\weights/weights-030-45.33.h5

Epoch 00030: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 31/200
60000/60000 [==============================] - 185s 3ms/step - loss: 45.3098 - vae_r_loss: 40.1807 - vae_kl_loss: 5.1291

Epoch 00031: saving model to run/vae/0002_digits\weights/weights-031-45.31.h5

Epoch 00031: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 32/200
60000/60000 [==============================] - 185s 3ms/step - loss: 45.2584 - vae_r_loss: 40.1211 - vae_kl_loss: 5.1373

Epoch 00032: saving model to run/vae/0002_digits\weights/weights-032-45.26.h5

Epoch 00032: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 33/200
60000/60000 [==============================] - 184s 3ms/step - loss: 45.2064 - vae_r_loss: 40.0702 - vae_kl_loss: 5.1362

Epoch 00033: saving model to run

Epoch 00058: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 59/200
60000/60000 [==============================] - 184s 3ms/step - loss: 44.4891 - vae_r_loss: 39.2439 - vae_kl_loss: 5.2453

Epoch 00059: saving model to run/vae/0002_digits\weights/weights-059-44.49.h5

Epoch 00059: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 60/200
60000/60000 [==============================] - 183s 3ms/step - loss: 44.4526 - vae_r_loss: 39.1954 - vae_kl_loss: 5.2572

Epoch 00060: saving model to run/vae/0002_digits\weights/weights-060-44.45.h5

Epoch 00060: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 61/200
60000/60000 [==============================] - 183s 3ms/step - loss: 44.4627 - vae_r_loss: 39.2194 - vae_kl_loss: 5.24320s - loss: 44.4593 - vae_r_loss: 39.2162 - vae_kl_loss: 5.

Epoch 00061: saving model to run/vae/0002_digits\weights/weights-061-44.46.h5

Epoch 00061: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 62/200
60000/60000 [==

60000/60000 [==============================] - 151s 3ms/step - loss: 44.0280 - vae_r_loss: 38.6982 - vae_kl_loss: 5.3297

Epoch 00087: saving model to run/vae/0002_digits\weights/weights-087-44.03.h5

Epoch 00087: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 88/200
60000/60000 [==============================] - 151s 3ms/step - loss: 44.0166 - vae_r_loss: 38.6783 - vae_kl_loss: 5.3383

Epoch 00088: saving model to run/vae/0002_digits\weights/weights-088-44.02.h5

Epoch 00088: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 89/200
60000/60000 [==============================] - 150s 2ms/step - loss: 44.0102 - vae_r_loss: 38.6783 - vae_kl_loss: 5.3318

Epoch 00089: saving model to run/vae/0002_digits\weights/weights-089-44.01.h5

Epoch 00089: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 90/200
60000/60000 [==============================] - 176s 3ms/step - loss: 43.9751 - vae_r_loss: 38.6463 - vae_kl_loss: 5.3288

Epoch 00090: saving model to run

60000/60000 [==============================] - 197s 3ms/step - loss: 43.7265 - vae_r_loss: 38.3437 - vae_kl_loss: 5.3828

Epoch 00116: saving model to run/vae/0002_digits\weights/weights-116-43.73.h5

Epoch 00116: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 117/200
60000/60000 [==============================] - 188s 3ms/step - loss: 43.7250 - vae_r_loss: 38.3328 - vae_kl_loss: 5.3922

Epoch 00117: saving model to run/vae/0002_digits\weights/weights-117-43.73.h5

Epoch 00117: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 118/200
60000/60000 [==============================] - 203s 3ms/step - loss: 43.6919 - vae_r_loss: 38.2970 - vae_kl_loss: 5.3949

Epoch 00118: saving model to run/vae/0002_digits\weights/weights-118-43.69.h5

Epoch 00118: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 119/200
60000/60000 [==============================] - 189s 3ms/step - loss: 43.6781 - vae_r_loss: 38.2794 - vae_kl_loss: 5.3986

Epoch 00119: saving model to 

60000/60000 [==============================] - 181s 3ms/step - loss: 43.5046 - vae_r_loss: 38.0738 - vae_kl_loss: 5.4308

Epoch 00145: saving model to run/vae/0002_digits\weights/weights-145-43.50.h5

Epoch 00145: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 146/200
60000/60000 [==============================] - 231s 4ms/step - loss: 43.5030 - vae_r_loss: 38.0816 - vae_kl_loss: 5.4214

Epoch 00146: saving model to run/vae/0002_digits\weights/weights-146-43.50.h5

Epoch 00146: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 147/200
60000/60000 [==============================] - 246s 4ms/step - loss: 43.4906 - vae_r_loss: 38.0549 - vae_kl_loss: 5.4357

Epoch 00147: saving model to run/vae/0002_digits\weights/weights-147-43.49.h5

Epoch 00147: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 148/200
60000/60000 [==============================] - 228s 4ms/step - loss: 43.4852 - vae_r_loss: 38.0375 - vae_kl_loss: 5.4477

Epoch 00148: saving model to 

60000/60000 [==============================] - 149s 2ms/step - loss: 43.3287 - vae_r_loss: 37.8566 - vae_kl_loss: 5.4722

Epoch 00174: saving model to run/vae/0002_digits\weights/weights-174-43.33.h5

Epoch 00174: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 175/200
60000/60000 [==============================] - 149s 2ms/step - loss: 43.2897 - vae_r_loss: 37.8145 - vae_kl_loss: 5.4752

Epoch 00175: saving model to run/vae/0002_digits\weights/weights-175-43.29.h5

Epoch 00175: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 176/200
60000/60000 [==============================] - 149s 2ms/step - loss: 43.3394 - vae_r_loss: 37.8722 - vae_kl_loss: 5.4671

Epoch 00176: saving model to run/vae/0002_digits\weights/weights-176-43.34.h5

Epoch 00176: saving model to run/vae/0002_digits\weights/weights.h5
Epoch 177/200
60000/60000 [==============================] - 165s 3ms/step - loss: 43.3056 - vae_r_loss: 37.8378 - vae_kl_loss: 5.4677

Epoch 00177: saving model to 

END.